In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from dotenv import load_dotenv

load_dotenv()

# Create a custom tool
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "gpt-4o-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 8192, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

In [ ]:
!python strands_agents_openai.py "{\"prompt\": \"海上升明月，下一句是什么\"}"

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

from boto3.session import Session
import boto3

import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

boto_session = Session()
region = boto_session.region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "openai-apikey-provider",
    "apiKey": openai_api_key # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
})

In [ ]:
%%writefile strands_agents_openai.py
import json
from strands import Agent, tool
from strands_tools import calculator
from strands.models.litellm import LiteLLMModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from botocore.exceptions import ClientError
from boto3.session import Session

boto_session = Session()

secret_name = "bedrock-agentcore-identity!default/apikey/openai-apikey-provider"

def get_secret(session="", secret_name=""):

    session = session
    secret_name = secret_name
    region_name = session.region_name

    # Create a Secrets Manager client

    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    secret = get_secret_value_response['SecretString']

    return secret
    # Your code goes here.

api_key_value = get_secret(boto_session, secret_name)

data = json.loads(api_key_value)
api_key = data["api_key_value"]

app = BedrockAgentCoreApp()

@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@app.entrypoint
async def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """

    # Initialize agent after API key is set
    print("Initializing agent with OPENAI API KEY...")
    model = "gpt-4o-mini"
    litellm_model = LiteLLMModel(
        model_id=model,
        params={
            "max_tokens": 8192,
            "temperature": 0.7
        },
        client_args={
            "api_key": api_key
        }
    )

    agent = Agent(
        model=litellm_model,
        tools=[calculator, weather],
        system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
    )
    print("Agent initialized successfully")

    user_input = payload.get("prompt")
    print(f"User input: {user_input}")

    try:
        response = agent(user_input)
        print(f"Agent response: {response}")
        return response.message['content'][0]['text']
    except Exception as e:
        print(f"Error in agent processing: {e}")
        raise

if __name__ == "__main__":
    app.run()

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()
agent_name="strands_agents_openai"

response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    region=region
)
response

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

In [ ]:
import json
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "arn:aws:secretsmanager:*:*:secret:bedrock-agentcore*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hello"}, user_id="userid_1234567890")
invoke_response

In [ ]:
from IPython.display import Markdown, display
response_text = invoke_response['response'][0]
display(Markdown(response_text))

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    runtimeUserId="userid_1234567890",
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))